##Model Training

In [1]:

import pandas as pd
import re
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier


df = pd.read_csv('/content/ODI.csv')

# fixing date ranges in our data
def fix_dates(date_str):
    # Using start date whenever we encounter a range
    match = re.match(r"([A-Za-z]+ [0-9]+)-[0-9]+, ([0-9]{4})", date_str)
    if match:
        return match.group(1) + ", " + match.group(2)
    return date_str

# Calling fix_date function on "Match Date" column and converting it to datetime format
df['Match Date'] = df['Match Date'].apply(fix_dates)
df['Match Date'] = pd.to_datetime(df['Match Date'], format='%b %d, %Y')

# Filtering the DataFrame for matches after the year 2000
df = df[df['Match Date'] > pd.to_datetime('2000-01-01')]

# We then rank the Match Date Column
df['date_rank'] = df['Match Date'].rank(ascending=False)

# Sample weights
df['sample_weight'] = 1 / df['date_rank']

# Removing rows with no winners
df = df[(df['Winner'] == df['Team_1']) | (df['Winner'] == df['Team_2'])]

# Keep relevant teams only
important_teams = [
    "India", "Australia", "South Africa", "Pakistan", "New Zealand",
    "England", "Sri Lanka", "Bangladesh", "Afghanistan", "West Indies",
    "Zimbabwe", "Scotland", "Ireland"
    #, "Nederlands", "Nepal",
    #"Namibia", "United States", "Oman", "UAE"
]
df = df[df['Team_1'].isin(important_teams) | df['Team_2'].isin(important_teams)]



# We give one hot encoding to the team names
team_1_encoded = pd.get_dummies(df['Team_1'], prefix='Team_1')
team_2_encoded = pd.get_dummies(df['Team_2'], prefix='Team_2')
df_encoded = pd.concat([df, team_1_encoded, team_2_encoded], axis=1)

# Unnecessary columns are dropped
#df_encoded.drop(columns=['Number', 'Team_1', 'Team_2', 'Margin', 'Ground', 'Match Date'], inplace=True)
df_encoded.drop(columns=['Number', 'Team_1', 'Team_2', 'Margin', 'Ground'], inplace=True)

# Divide the dataset into training data and test data

"""
X = df_encoded.drop(columns=['Winner'])


y = df_encoded['Winner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

"""

# Define a cutoff date for splitting the data
cutoff_date = pd.to_datetime('2019-01-01')

# Splitting the data into training and testing sets based on the 'Match Date'
X_train = df_encoded[df_encoded["Match Date"] < cutoff_date].drop(columns=['Winner', 'Match Date'])
X_test = df_encoded[df_encoded["Match Date"] >= cutoff_date].drop(columns=['Winner', 'Match Date'])

# Splitting the target variable ('Winner') accordingly
y_train = df_encoded[df_encoded["Match Date"] < cutoff_date]['Winner']
y_test = df_encoded[df_encoded["Match Date"] >= cutoff_date]['Winner']




# Next up we create a dictionary of classifiers
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=5000, C=100, penalty='l2'),
    #"SVM": SVC(),
    #"KNN": KNeighborsClassifier(),
    #"Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    #"Random Forest": RandomForestClassifier(n_estimators = 200, min_samples_split = 10, min_samples_leaf = 4, max_depth = 30),
    "Random Forest": RandomForestClassifier(n_estimators = 40, min_samples_split = 10, random_state = 1),
    "Neural Network": MLPClassifier(max_iter=5000)

}

models = {}
results = {}
# Looping over different classifiers
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    results[name] = score
    models[name] = clf  # Save the model
    print(f"{name}: {score:.4f}")


best_classifier = max(results, key=results.get)
print(f"\nThe best classifier is {best_classifier} with an accuracy of {results[best_classifier]:.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: 0.7198
Gradient Boosting: 0.6676
Random Forest: 0.6456
Neural Network: 0.6676

The best classifier is Logistic Regression with an accuracy of 0.7198


## Merging of Two Datasets

In [3]:
import pandas as pd

# Original Dataset

df2 = pd.read_csv('/content/ODI.csv')


# Odds dataset from XLSX file
odds_df = pd.read_excel('/content/BettingOdds.xlsx')


# Now we have two dataframes: df2 for the model's predictions and odds_df for the betting odds

import pandas as pd
import numpy as np
import re
import requests

# fixing date ranges in our data
def fix_dates(date_str):
    # Using start date whenever we encounter a range
    match = re.match(r"([A-Za-z]+ [0-9]+)-[0-9]+, ([0-9]{4})", date_str)
    if match:
        return match.group(1) + ", " + match.group(2)
    return date_str

# Calling fix_date function on "Match Date" column and converting it to datetime format
df2['Match Date'] = df2['Match Date'].apply(fix_dates)
df2['Match Date'] = pd.to_datetime(df2['Match Date'], format='%b %d, %Y')

# a) We than convert the date format in odds_df to datetime format (having the format is like '27 Sep 2023')
#odds_df['Date'] = pd.to_datetime(odds_df['Date'], format='%d %b %Y')

# For the odds dataset, adjustment to the formatt
# odds_df['Date'] = pd.to_datetime(odds_df['Date'], format='%d %b %Y,')



def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%d %b %Y,')
    except ValueError:
        # Return NaT (Not a Time) for unparseable dates
        return pd.NaT

# Applying this function to the date column
odds_df['Date'] = odds_df['Date'].apply(parse_date)

# Dropoing rows where 'Date' is NaT
odds_df.dropna(subset=['Date'], inplace=True)



# b) Missing values removed
odds_df = odds_df.dropna()

# c) Extract floats from the 'Bet1' and 'Bet2' columns
odds_df['Bet1'] = odds_df['Bet1'].str.extract('(\d+\.\d+|\d+)').astype(float)
odds_df['Bet2'] = odds_df['Bet2'].str.extract('(\d+\.\d+|\d+)').astype(float)

# d) Check if team names correspond between df2 and odds_df based on the date
# For simplicity, let's assume df2 has been filtered to only include dates that are present in odds_df
# And that df2 is sorted in the same order as odds_df
# This will simply add a 'Match' column that says 'Yes' if both team names match, 'No' otherwise
#df2['Match'] = np.where((df2['Team_1'] == odds_df['Team1']) & (df2['Team_2'] == odds_df['Team2']), 'Yes', 'No')

# First, ensure that both dataframes have the date in the same format
df2['Match Date'] = pd.to_datetime(df2['Match Date'], format='%b %d, %Y')
odds_df['Date'] = pd.to_datetime(odds_df['Date'], format='%d %b %Y')



# Now perform an inner merge to align the rows based on the date
# This will combine rows where the dates match
merged_df = pd.merge(df2, odds_df, left_on='Match Date', right_on='Date')
# Merging the datasets
#merged_df = pd.merge(df2, odds_df, left_on='Match Date', right_on='Date', how='inner')
# After merging, you can then compare the team names
"""
merged_df['Match'] = np.where((merged_df['Team_1'] == merged_df['Team1']) &
                              (merged_df['Team_2'] == merged_df['Team2']), 'Yes', 'No')
"""

# Function to check if teams match (regardless of order)
def teams_match(row):
    teams_df2 = {row['Team_1'], row['Team_2']}
    teams_odds = {row['Team1'], row['Team2']}
    return teams_df2 == teams_odds

# Apply the function to each row in the merged dataframe
merged_df['Match'] = merged_df.apply(teams_match, axis=1).map({True: 'Yes', False: 'No'})

# Calculate the percentage of 'Yes' in the 'Match' column
match_percentage = (merged_df['Match'].value_counts(normalize=True) * 100).get('Yes', 0)
print(f"Percentage of correctly matched rows: {match_percentage:.2f}%")

"""
# Step 1: Prepare the keys for matching
# Creating a sorted list of team names for each row
df2['team_pair'] = df2.apply(lambda row: '_'.join(sorted([row['Team_1'], row['Team_2']])), axis=1)
odds_df['team_pair'] = odds_df.apply(lambda row: '_'.join(sorted([row['Team1'], row['Team2']])), axis=1)

# Combining the date and team pair to form a unique key
df2['match_key'] = df2['Match Date'].astype(str) + '_' + df2['team_pair']
odds_df['match_key'] = odds_df['Date'].astype(str) + '_' + odds_df['team_pair']

# Step 2: Merge based on the match_key
merged_df = pd.merge(df2, odds_df, on='match_key', how='inner')

# Step 3: Evaluate the match percentage
match_percentage = len(merged_df) / min(len(df2), len(odds_df)) * 100
print(f"Percentage of correctly matched rows: {match_percentage:.2f}%")

# Optionally, you can drop the extra columns created for matching
merged_df.drop(['team_pair', 'match_key'], axis=1, inplace=True)

# Step 4: Continue with your further analysis or processing


"""

print(merged_df[merged_df['Match'] == 'No'])


# If you only want to keep rows where the team names match:
matched_df = merged_df[merged_df['Match'] == 'Yes']


# e) Split the merged_df dataset for training and backtesting
odds_train = merged_df[(merged_df['Date'].dt.year >= 2009) & (merged_df['Date'].dt.year <= 2021)]
odds_test = merged_df[(merged_df['Date'].dt.year >= 2022) & (merged_df['Date'].dt.year <= 2023)]


Percentage of correctly matched rows: 72.65%
      Number        Team_1       Team_2        Winner     Margin  \
18      2868      Zimbabwe   Bangladesh    Bangladesh  8 wickets   
21      2869     Sri Lanka     Pakistan      Pakistan   132 runs   
36      2916    Bangladesh     Zimbabwe    Bangladesh  4 wickets   
56      2949     Australia  West Indies     Australia   113 runs   
70      2968   New Zealand    Australia     Australia    12 runs   
...      ...           ...          ...           ...        ...   
1113    4548      Zimbabwe  Netherlands      Zimbabwe  7 wickets   
1114    4556   New Zealand    Sri Lanka   New Zealand  6 wickets   
1117    4557  South Africa  Netherlands  South Africa  8 wickets   
1118    4560        P.N.G.       U.S.A.        U.S.A.   117 runs   
1119    4561       Namibia       U.A.E.        U.A.E.    28 runs   

             Ground Match Date         Team1        Team2  Bet1   Bet2  \
18         Bulawayo 2009-08-09     Sri Lanka     Pakistan  1.72 

<ipython-input-3-d063b9610033>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds_df['Bet1'] = odds_df['Bet1'].str.extract('(\d+\.\d+|\d+)').astype(float)
<ipython-input-3-d063b9610033>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds_df['Bet2'] = odds_df['Bet2'].str.extract('(\d+\.\d+|\d+)').astype(float)
<ipython-input-3-d063b9610033>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [4]:
merged_df

,Number,Team_1,Team_2,Winner,Margin,Ground,Match Date,Team1,Team2,Bet1,Bet2,Result,Date,Match
0,2824,New Zealand,India,New Zealand,8 wickets,Auckland,2009-03-14,New Zealand,India,2.62,1.44,New Zealand won by 8 wickets.\n151/2 (23.2) : ...,2009-03-14,Yes
1,2825,West Indies,England,England,1 run,Providence,2009-03-20,West Indies,England,1.86,1.82,England won by 1 run (DLS method).\n244/7 (46....,2009-03-20,Yes
2,2833,South Africa,Australia,South Africa,7 wickets,Centurion,2009-04-05,South Africa,Australia,1.89,1.89,South Africa won by 7 wickets.\n132/3 (26.2) :...,2009-04-05,Yes
3,2838,South Africa,Australia,South Africa,25 runs,Cape Town,2009-04-09,South Africa,Australia,1.71,2.10,South Africa won by 25 runs.\n289/6 (50) : 264...,2009-04-09,Yes
4,2839,South Africa,Australia,South Africa,61 runs,Gqeberha,2009-04-13,South Africa,Australia,1.76,2.00,South Africa won by 61 runs.\n317/6 (50) : 256...,2009-04-13,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,4573,Pakistan,New Zealand,Pakistan,102 runs,Karachi,2023-05-05,Pakistan,New Zealand,1.44,2.81,Pakistan won by 102 runs.,2023-05-05,Yes
1122,4574,Pakistan,New Zealand,New Zealand,47 runs,Karachi,2023-05-07,Pakistan,New Zealand,1.42,2.89,New Zealand won by 47 runs.,2023-05-07,Yes
1123,4575,Bangladesh,Ireland,no result,-,Chelmsford,2023-05-09,Ireland,Bangladesh,2.59,1.50,No result.,2023-05-09,Yes
1124,4576,Bangladesh,Ireland,Bangladesh,3 wickets,Chelmsford,2023-05-12,Bangladesh,Ireland,1.49,2.63,,2023-05-12,Yes


In [5]:
# We then rank the Match Date Column
merged_df['date_rank'] = merged_df['Match Date'].rank(ascending=False)

# Sample weights based on recency of data
merged_df['sample_weight'] = 1 / merged_df['date_rank']

merged_df_encoded = pd.concat([merged_df], axis=1)

# We give one hot encoding to the team names
team_1_encoded = pd.get_dummies(merged_df['Team_1'], prefix='Team_1')
team_2_encoded = pd.get_dummies(merged_df['Team_2'], prefix='Team_2')
merged_df_encoded = pd.concat([merged_df, team_1_encoded, team_2_encoded], axis=1)


# Removing rows with no winners
merged_df_encoded = merged_df_encoded[(merged_df_encoded['Winner'] == merged_df_encoded['Team_1']) | (merged_df_encoded['Winner'] == merged_df_encoded['Team_2'])]

# Keep relevant teams only
important_teams = [
    "India", "Australia", "South Africa", "Pakistan", "New Zealand",
    "England", "Sri Lanka", "Bangladesh", "Afghanistan", "West Indies",
    "Zimbabwe", "Scotland", "Ireland"
    #, "Nederlands", "Nepal",
    #"Namibia", "United States", "Oman", "UAE"
]
merged_df_encoded = merged_df_encoded[merged_df_encoded['Team_1'].isin(important_teams) | merged_df_encoded['Team_2'].isin(important_teams)]

In [6]:
merged_df_encoded

,Number,Team_1,Team_2,Winner,Margin,Ground,Match Date,Team1,Team2,Bet1,...,Team_2_Oman,Team_2_P.N.G.,Team_2_Pakistan,Team_2_Scotland,Team_2_South Africa,Team_2_Sri Lanka,Team_2_U.A.E.,Team_2_U.S.A.,Team_2_West Indies,Team_2_Zimbabwe
0,2824,New Zealand,India,New Zealand,8 wickets,Auckland,2009-03-14,New Zealand,India,2.62,...,0,0,0,0,0,0,0,0,0,0
1,2825,West Indies,England,England,1 run,Providence,2009-03-20,West Indies,England,1.86,...,0,0,0,0,0,0,0,0,0,0
2,2833,South Africa,Australia,South Africa,7 wickets,Centurion,2009-04-05,South Africa,Australia,1.89,...,0,0,0,0,0,0,0,0,0,0
3,2838,South Africa,Australia,South Africa,25 runs,Cape Town,2009-04-09,South Africa,Australia,1.71,...,0,0,0,0,0,0,0,0,0,0
4,2839,South Africa,Australia,South Africa,61 runs,Gqeberha,2009-04-13,South Africa,Australia,1.76,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,4562,South Africa,Netherlands,South Africa,146 runs,Johannesburg,2023-04-02,South Africa,Netherlands,1.01,...,0,0,0,0,0,0,0,0,0,0
1121,4573,Pakistan,New Zealand,Pakistan,102 runs,Karachi,2023-05-05,Pakistan,New Zealand,1.44,...,0,0,0,0,0,0,0,0,0,0
1122,4574,Pakistan,New Zealand,New Zealand,47 runs,Karachi,2023-05-07,Pakistan,New Zealand,1.42,...,0,0,0,0,0,0,0,0,0,0
1124,4576,Bangladesh,Ireland,Bangladesh,3 wickets,Chelmsford,2023-05-12,Bangladesh,Ireland,1.49,...,0,0,0,0,0,0,0,0,0,0


## Betting Strategies

In [7]:
# Part I

# Task 1: Calculating Implied Probability
merged_df_encoded['Implied_Prob_Team1'] = 1 / merged_df_encoded['Bet1']
merged_df_encoded['Implied_Prob_Team2'] = 1 / merged_df_encoded['Bet2']

# Task 2: Model's Predicted Probability using the best classifier from model training
best_classifier_name = max(results, key=results.get)
best_classifier_model = models[best_classifier_name]

# Probability Prediciton
# List all columns that are in X_train but not in merged_df_encoded
missing_cols = [col for col in X_train.columns if col not in merged_df_encoded.columns]

# Add these missing columns to merged_df_encoded with a default value of 0
for col in missing_cols:
    merged_df_encoded[col] = 0

# Now, merged_df_encoded should have the same columns as X_train
# We can now proceed with the prediction
probabilities = best_classifier_model.predict_proba(merged_df_encoded[X_train.columns])

probabilities = best_classifier_model.predict_proba(merged_df_encoded[X_train.columns])
merged_df_encoded['Model_Prob_Team1'] = probabilities[:, 0]
merged_df_encoded['Model_Prob_Team2'] = probabilities[:, 1]

# Task 3: Finding Value Bets
merged_df_encoded['Value_Bet_Team1'] = merged_df_encoded['Model_Prob_Team1'] > merged_df_encoded['Implied_Prob_Team1']
merged_df_encoded['Value_Bet_Team2'] = merged_df_encoded['Model_Prob_Team2'] > merged_df_encoded['Implied_Prob_Team2']

# Task 4: Backtesting (Simulating betting based on these predictions)

# We 'Bet_On_Team1' if Value_Bet_Team1 condition is satisfied and actual Winner is also Team1 :)
merged_df_encoded['Bet_On_Team1'] = merged_df_encoded['Value_Bet_Team1'] & (merged_df_encoded['Winner'] == merged_df_encoded['Team_1'])
merged_df_encoded['Bet_On_Team2'] = merged_df_encoded['Value_Bet_Team2'] & (merged_df_encoded['Winner'] == merged_df_encoded['Team_2'])

# Profit/Loss Calculation

# recall np.where(condition, x, y): if conditon is satisfied x, else y
# for simplicity we base calculations based on unit bet (1), i.e  net profit per unit staked
merged_df_encoded['Profit_Team1'] = np.where(merged_df_encoded['Bet_On_Team1'], merged_df_encoded['Bet1'] - 1, 0)
merged_df_encoded['Profit_Team2'] = np.where(merged_df_encoded['Bet_On_Team2'], merged_df_encoded['Bet2'] - 1, 0)
merged_df_encoded['Total_Profit'] = merged_df_encoded['Profit_Team1'] + merged_df_encoded['Profit_Team2']

# Total Profit oder Loss
total_profit = merged_df_encoded['Total_Profit'].sum()
print(f"Total Profit from Backtesting: {total_profit}")


Total Profit from Backtesting: 117.42


## Kelly Criterion

In [11]:

def kelly_criterion(b, p): # This function calculates the Kelly Fraction, which determines the optimal portion of your bankroll to betWe consider again unit bets and the rest of the odds as potential profit. So, b is set as decimal odds - 1 to represent the net profit per unit of stake, excluding the original stake. If the decimal odds are 2.0, your net profit for a winning bet is 2.0 - 1 = 1 unit per unit staked, not 2 units.
    q = 1 - p
    kelly_fraction = (b * p - q) / b
    return max(kelly_fraction, 0)  # Ensure no negative bets

# Assuming 'Bet1' and 'Bet2' are decimal odds
merged_df_encoded['Kelly_Fraction_Team1'] = merged_df_encoded.apply(lambda row: kelly_criterion(row['Bet1'] - 1, row['Model_Prob_Team1']), axis=1)
merged_df_encoded['Kelly_Fraction_Team2'] = merged_df_encoded.apply(lambda row: kelly_criterion(row['Bet2'] - 1, row['Model_Prob_Team2']), axis=1)

# Assuming a fixed bankroll for each bet
bankroll = 100  # example bankroll for each bet

merged_df_encoded['Bet_Amount_Team1'] = merged_df_encoded['Kelly_Fraction_Team1'] * bankroll
merged_df_encoded['Bet_Amount_Team2'] = merged_df_encoded['Kelly_Fraction_Team2'] * bankroll
# By multiplying this fraction by our total bankroll for each bet (bankroll * Kelly_Fraction), we are determining the optimal bet size as recommended by the Kelly Criterion.

# We adjust profit/loss calculation
merged_df_encoded['Profit_Team1'] = np.where(merged_df_encoded['Bet_On_Team1'], merged_df_encoded['Bet_Amount_Team1'] * (merged_df_encoded['Bet1'] - 1), -merged_df_encoded['Bet_Amount_Team1']) # -merged_df_encoded['Bet_Amount_Team1 would eventually mean 0, since we cap it to 0
merged_df_encoded['Profit_Team2'] = np.where(merged_df_encoded['Bet_On_Team2'], merged_df_encoded['Bet_Amount_Team2'] * (merged_df_encoded['Bet2'] - 1), -merged_df_encoded['Bet_Amount_Team2'])
merged_df_encoded['Total_Profit'] = merged_df_encoded['Profit_Team1'] + merged_df_encoded['Profit_Team2']

# Calculating total profit or loss
total_profit = merged_df_encoded['Total_Profit'].sum()
print(f"Total Profit from Backtesting with Kelly Criterion: {total_profit}")

# We can also calculate other metrics, such as the total amount bet and the number of bets made
total_bet = merged_df_encoded['Bet_Amount_Team1'].sum() + merged_df_encoded['Bet_Amount_Team2'].sum()
number_of_bets = (merged_df_encoded['Bet_Amount_Team1'] > 0).sum() + (merged_df_encoded['Bet_Amount_Team2'] > 0).sum()

print(f"Total Amount Bet: {total_bet}")
#print(f"Number of Bets Made: {number_of_bets}")


Total Profit from Backtesting with Kelly Criterion: 1262.4249396103892
Total Amount Bet: 7168.726984743158
